# Ordinal SeqDEFT

This is a demo of the density estimation method, **Ordinal SeqDEFT** (standing for Ordinal Sequence Density Estimation using Field Theory). It includes the following subjects:

1. Preliminary preparation
2. Data importation
3. MAP estimation
4. Cross validation
5. Getting the result
6. Computing associations
7. Making visualization

As an example, we use it to compute a small problem that involves only 27 sequences. For problems with tens of thousands of sequences, the calculation can be readily done on a typical laptop computer. For problems larger than that, deploying the computations on a workstation or a cluster can greatly facilitate the calculation. Even with parallel computation, however, the maximum number of sequences that the algorithm can run on is about a few millions.

**Reference**
- Wei-Chia Chen, Juannan Zhou, and David M. McCandlish, [Density estimation for ordinal biological sequences and its applications](https://arxiv.org/abs/2404.11228), arXiv:2404.11228 (2024)

Import Python packages that will be used.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Import Ordinal SeqDEFT functions from the file `functions.py`.

In [ ]:
from functions import preliminary_preparation, import_data, trace_MAP_curve, \
                      compute_log_likelihoods, make_visualization, get_nodes, get_edges, \
                      compute_rms_log_p_association, compute_log_p_associations

Suppose we have a set of ordinal sequence data. Each sequence has 3 sites, and each site can take on 3 possible elements, say, `1`, `2`, `3`. In ordinal SeqDEFT, the sequence space is represented by a 3 x 3 x 3 grid graph, as shown below:

<img src='grid.png' height='350' width='350'>

In the figure each node corresponds to a sequence, and each edge corresponds to a single point mutation. The number of times that a sequence was observed is represented by the size of the sphere. The data is stored in the file `data.txt`. Our goal is to estimate the probability distribution from which the sequences were drawn.

### 1. Preliminary preparation
Specify number of elements `alpha`, number of sites `l`, and order of the smoothness operator `P` with the function 
``` python
preliminary_preparation(alpha, l, P, parameters_only=False, time_it=False)
```
This will set up the stuff that will be needed throughout the calculation, including kernel basis of the smoothness operator.

In [ ]:
alpha, l, P = 3, 3, 2

# ---

preliminary_preparation(alpha, l, P)

### 2. Data importation
Import data with the function
```python
import_data(path, coding_dict, ignore_sites=None)
```
Data should be prepared in a count-data format as the example file `data.txt`, in which sequences are stored in the first column and counts are stored in the second column, separated by one or more whitespace characters. Unobserved sequences need not be listed in the file. Also, we should specify how to encode elements. In Ordinal SeqDEFT, elements from small to large are encoded as `0`, `1`, `2`, ... and so on.

In [ ]:
path = 'data.txt'
coding_dict = {'1':0, '2':1, '3':2}

# ---

data_dict = import_data(path, coding_dict)

The function will return a dictionary `data_dict` that contains total number of sequences `N` in the data and frequencies of each possible sequence `R`.

In [ ]:
data_dict

### 3. MAP estimation
For each value of the hyperparameter $a$ between zero and infinity, find the corresponding MAP solution of $\phi$. This is done with the function
```python
trace_MAP_curve(data_dict, resolution=0.1, num_a=20, fac_max=1, fac_min=1e-3, options=None, scale_by=1)
```
The function will first find a large value of $a$ to substitute infinity and a small value of $a$ to substitute zero. This is achieved by increasing (decreasing) the value of $a$ from `s * fac_max` (`s * fac_min`), where `s` is the number of $P$-dimensional hypercubes, until the geodesic distance between the corresponding distribution and $Q_\infty$ ($Q_0$) is less than `resolution`. Then `num_a` values of $a$, including the two extremes found above, are selected by a geometric partition. These values, together with zero and infinity, represent the entire MAP curve.

Also, we can control the minimization procedure by passing parameters through the argument `options`. From our experience, using gradient (controlled by `gtol`) rather than function value (controlled by `ftol`) as the stopping criterion is safer as the latter may result in a premature termination. The stopping criterion based on function value can be "turned off" by setting the parameter `ftol` to an extremely small value.

In [ ]:
num_a = 20
options = {'ftol':1e-100, 'gtol':1e-5, 'maxiter':50000, 'maxfun':100000}

# ---

df_map = trace_MAP_curve(data_dict, num_a=num_a, options=options)

The function will return a dataframe `df_map` that contains the values of $a$ and the corresponding field $\phi_a$.

In [ ]:
df_map.head()

One way to check if the MAP solutions $\phi_a$'s were solved accurately is to see if they satisfy the condition: $\sum e^{-\phi_a} = 1$.

In [ ]:
for i in range(len(df_map)):
    a, phi = df_map.loc[i, ['a','phi']]
    print('sum = %f at a = %.2f' % (np.sum(np.exp(-phi)), a))

### 4. Cross validation
To find the optimal value of $a$, we employ $k$-fold cross validation. This is done by the function
```python
compute_log_likelihoods(data_dict, df_map, cv_fold=5, random_seed=None, options=None, scale_by=1)
```
As in the part of MAP estimation, the argument `options` is used to pass parameters controlling the minimization procedure.

In [ ]:
cv_fold = 5
random_seed = 0
options = {'ftol':1e-100, 'gtol':1e-5, 'maxiter':50000, 'maxfun':100000}

# ---

df_map = compute_log_likelihoods(data_dict, df_map, cv_fold, random_seed, options)

The function will return the same dataframe `df_map` with an additional column recording the cross-validated log likelihoods.

In [ ]:
df_map.head()

### 5. Getting the result
Plot cross-validated log likelihood $\log L$ versus the hyperparameter $a$. Note that $\log L = -\infty$ at $a = 0$.

In [ ]:
aa, log_Ls = df_map['a'].values, df_map['log_L'].values
i_star = log_Ls.argmax()
a_star, max_log_L = aa[i_star], log_Ls[i_star]

# ---

plt.figure(figsize=(6,4.5))

with np.errstate(divide='ignore'):
    plt.scatter(np.log10(aa), log_Ls, color='blue', zorder=1)
    plt.scatter(4, log_Ls[-1], color='blue', zorder=1)
    plt.scatter(np.log10(a_star), max_log_L, color='red', zorder=2, label=r'$a^*$ = %.1f'%a_star)

plt.xlim(-2, 4)
plt.ylim(-71, -66)
plt.xticks([-2, -1, 0, 1, 2, 3, 4], ['-2', '-1', '0', '1', '2', '3', r'$\infty$'], fontsize=16)
plt.yticks([-71, -70, -69, -68, -67, -66], ['-71', '-70', '-69', '-68', '-67', '-66'], fontsize=16)
plt.xlabel(r'$\log_{10} (a)$', fontsize=20)
plt.ylabel(r'$\log \ \! (L)$', fontsize=20)
plt.legend(fontsize=16)
plt.show()

From the plot we can see that the optimal value of $a$ is about $1.1$, and the corresponding distribution, denoted $Q^*$, is what we want.

We can compare $Q^*$ with the observed frequency $Q_0$.

In [ ]:
phis = df_map['phi'].values
phi_star = phis[i_star]
Q_star = np.exp(-phi_star) / np.sum(np.exp(-phi_star))
phi_0 = phis[0]
Q_0 = np.exp(-phi_0) / np.sum(np.exp(-phi_0))

# ---

plt.figure(figsize=(5.5,5.5))
plt.plot([0, 0.105], [0, 0.105], color='grey', linewidth=1, linestyle='--', zorder=1)
plt.scatter(Q_0, Q_star, color='blue', s=50, alpha=0.3, zorder=2)
plt.xlim(0, 0.105)
plt.ylim(0, 0.105)
plt.xticks([0, 0.02, 0.04, 0.06, 0.08, 0.10], ['0', '0.02', '0.04', '0.06', '0.08', '0.10'], fontsize=16)
plt.yticks([0, 0.02, 0.04, 0.06, 0.08, 0.10], ['0', '0.02', '0.04', '0.06', '0.08', '0.10'], fontsize=16)
plt.xlabel(r'$Q_0$', fontsize=20)
plt.ylabel(r'$Q$*', fontsize=20)
plt.show()

Looks great! 

On the other hand, the maximum entropy estimate $Q_\infty$ (which is an additive model in this case) gives bad results.

In [ ]:
phis = df_map['phi'].values
phi_inf = phis[-1]
Q_inf = np.exp(-phi_inf) / np.sum(np.exp(-phi_inf))
phi_0 = phis[0]
Q_0 = np.exp(-phi_0) / np.sum(np.exp(-phi_0))

# ---

plt.figure(figsize=(5.5,5.5))
plt.plot([0, 0.105], [0, 0.105], color='grey', linewidth=1, linestyle='--', zorder=1)
plt.scatter(Q_0, Q_inf, color='blue', s=50, alpha=0.3, zorder=2)
plt.xlim(0, 0.105)
plt.ylim(0, 0.105)
plt.xticks([0, 0.02, 0.04, 0.06, 0.08, 0.10], ['0', '0.02', '0.04', '0.06', '0.08', '0.10'], fontsize=16)
plt.yticks([0, 0.02, 0.04, 0.06, 0.08, 0.10], ['0', '0.02', '0.04', '0.06', '0.08', '0.10'], fontsize=16)
plt.xlabel(r'$Q_0$', fontsize=20)
plt.ylabel(r'$Q_\infty$', fontsize=20)
plt.show()

### 6. Computing associations
We can study associations among the sequence sites with the inferred probability distribution $Q^*$. 

First, we can compute the "association specturm" which tells us the association strength of each order. This is done with the function
```python
compute_rms_log_p_association(phi, p)
```
with $p = 1, 2, \dots, \ell$.

In [ ]:
rms_log_Aps = np.zeros(l)
for p in range(1, l+1):
    rms_log_Ap = compute_rms_log_p_association(phi_star, p)
    rms_log_Aps[p-1] = rms_log_Ap
    
# ---

plt.figure(figsize=(6,4.5))
plt.plot(range(1,l+1), np.exp(rms_log_Aps), color='blue')
plt.scatter(range(1,l+1), np.exp(rms_log_Aps), color='blue')
plt.xlim(0.75, 3.25)
plt.ylim(0, 30)
plt.xticks([1, 2, 3], ['1', '2', '3'], fontsize=16)
plt.yticks([0, 5, 10, 15, 20, 25, 30], ['0', '5', '10', '15', '20', '25', '30'], fontsize=16)
plt.xlabel(r'Order, $p$', fontsize=20)
plt.ylabel(r'Effective $A^{(p)}$', fontsize=20)
plt.show()

Each value of effective $A^{(p)}$ in the plot is equal to the exponential of the root-mean-square value of the $\log A^{(p)}$'s conditional on all possible backgrounds.

On the other hand, the function
```python
compute_log_p_associations(phi, sites_dict, condition_dict={}, coding_dict=None)
```
allows us to look into the associations. 

For example, to compute the association between the 1st site and the 2nd site (without specifying "mutations" at them), we do

In [ ]:
sites_dict = {0: None, # 1st site, with all possible mutations
              1: None} # 2nd site, with all possible mutations

# ---

df_log_Aps = compute_log_p_associations(phi_star, sites_dict, coding_dict={'1':0, '2':1, '3':2})

The function will return a dataframe `df_log_Aps`, recording the values of $\log A^{(p)}$ along with the associated sequences.

In [ ]:
df_log_Aps.head()

To compute the association between the 1st site and the 2nd site with the former "mutating" from `1` to `2` and the latter "mutating" from `2` to `3`, we do

In [ ]:
sites_dict = {0: ['1', '2'], # 1st site, mutating from '1' to '2'
              1: ['2', '3']} # 2nd site, mutating from '2' to '3'

# ---

df_log_Aps = compute_log_p_associations(phi_star, sites_dict, coding_dict={'1':0, '2':1, '3':2})
df_log_Aps

We can even specify the backgrounds in which the "mutations" take place. Let us use the same example and fix the 3rd site at `1` and `3`.

In [ ]:
sites_dict = {0: ['1', '2'], # 1st site, mutating from '1' to '2'
              1: ['2', '3']} # 2nd site, mutating from '2' to '3'
condition_dict = {2: ['1', '3']} # 3rd site, fixed at '1' and '3'

# ---

df_log_Aps = compute_log_p_associations(phi_star, sites_dict, condition_dict, coding_dict={'1':0, '2':1, '3':2})
df_log_Aps

In a similar manner, the function can be used to compute associations of any order, with or without specifying "mutations", conditional on certain backgrounds or not.

### 7. Making visualization
There are various dimensionality reduction techniques that can be used to visualize high-dimensional objects like $Q^*$. We employ the one based on an evolutionary model from [D. M. McCandlish (2011)](https://academic.oup.com/evolut/article/65/6/1544/6854390). This is done with the function
```python
make_visualization(Q, markov_chain, K=20, tol=1e-9, reuse_Ac=False, path='sparse_matrix/Ac/')
```
The input distribution `Q` will be used to construct the rate matrix of an imaginary Markov chain, whose first `K` eigenvectors will be solved and used as visualization coordinates. The accuracy of this computation is controlled by the parameter `tol`. The results are stored in two dataframes `df_visual` and `df_check`.

In [ ]:
Q = Q_star
markov_chain = 'evolutionary'
K = 10

# ---

df_visual, df_check = make_visualization(Q, markov_chain, K)

The dataframe `df_visual` stores the visualization coordinates. Note that the zeroth eigenvalue is supposed to be zero and the corresponding coordinates are supposed to be NAN's; both will not be used.

In [ ]:
df_visual.head()

The dataframe `df_check` contains information about the computation. In general, `colinearity` should be $1.0$ and `max_difference` should be as small as possible, say, around $10^{-16}$.

In [ ]:
df_check.head()

The reciprocal of the absolute value of each eigenvalue in `df_visual` is proportional to the variance explained in that direction. So let us first take a look at the spectrum.

In [ ]:
x, y = range(1, K), 1/abs(df_visual['eigenvalue'].values[1:])

# ---

plt.figure(figsize=(6,4.5))
plt.scatter(x, y, color='blue')
plt.xlim(0, K)
plt.ylim(0, )
plt.xticks([0, 2, 4, 6, 8, 10], ['0', '2', '4', '6', '8', '10'], fontsize=16)
plt.yticks([0, 0.5, 1, 1.5, 2, 2.5, 3], ['0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0'], fontsize=16)
plt.xlabel(r'$k$', fontsize=20)
plt.ylabel('1 / |Eigenvalue|', fontsize=20)
plt.show()

It seems reasonable to visualize $Q^*$ with the first two or three directions. To do that, we need to get the nodes and edges with the functions
```python
get_nodes(df_visual, kx, ky, xflip=1, yflip=1)
get_edges(df_visual, kx, ky, xflip=1, yflip=1)
```
The parameters `xflip` and `yflip` can be used to deform the graph via transformations `x = x * xflip` and `y = y * yflip`.

In [ ]:
kx, ky = 1, 2

# ---

df_nodes = get_nodes(df_visual, kx, ky)
df_edges = get_edges(df_visual, kx, ky)

Now we can make the plot. We color-code each node with the inferred probability of the corresponding sequence.

In [ ]:
from matplotlib.collections import LineCollection

color_by = np.log10(Q_star) 
vmin, vmax = np.floor(np.log10(Q_star).min()), np.ceil(np.log10(Q_star).max())

# ---

plt.figure(figsize=(8,6.5))

# Plot edges
edges = df_edges['edge'].values
ln_coll = LineCollection(edges, color='grey', linewidths=1, alpha=0.8, zorder=1)
ax = plt.gca()
ax.add_collection(ln_coll)
plt.draw()

# Plot nodes
nodes_x, nodes_y, nodes_c = df_nodes['x'].values, df_nodes['y'].values, color_by
plt.scatter(nodes_x, nodes_y, c=nodes_c, vmin=vmin, vmax=vmax, cmap='jet', s=30, zorder=2)
cbar = plt.colorbar()
cbar.set_label(r'$\log_{10} \ \! (Q^*)$', size=20)

plt.xlabel('Diffusion Axis %d'%kx, fontsize=20)
plt.ylabel('Diffusion Axis %d'%ky, fontsize=20)
plt.show()